In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math, re, os, random
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

print("TF version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

# Dataset Structure

⚙️ 2. TPU / GPU Setup

We automatically detect TPU or fallback to GPU:

In [ ]:
%%time
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except ValueError:
    strategy = tf.distribute.get_strategy()  # CPU or GPU
print("REPLICAS:", strategy.num_replicas_in_sync)

# Data access and classes¶

TPUs read data directly from Google Cloud Storage (GCS), so we need to copy the dataset to a GCS bucket co-located with the TPU. To do that, pass the name of a specific dataset to the get_gcs_path function. The name of the dataset is the name of the directory it is mounted in.

In [ ]:
%%time
import tensorflow as tf
from kaggle_datasets import KaggleDatasets

# ====================================================
# CONFIGURATION
# ====================================================
IMAGE_SIZE = [224, 224]   # You can switch to 331 or 512 later
BATCH_SIZE = 16

# ====================================================
# DATA PATHS
# ====================================================
try:
    # Kaggle-only: detect TPU and load dataset path from GCS
    GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
    USING_KAGGLE_GCS = True
    print("✅ Using Kaggle GCS dataset")
except Exception as e:
    # Local or GPU runtime
    GCS_DS_PATH = "/kaggle/input"
    USING_KAGGLE_GCS = False
    print("⚠️ Using local dataset path:", GCS_DS_PATH)

# Map image size → folder
GCS_PATH_SELECT = {
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512',
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

# Define TFRecord files
TRAINING_FILENAMES   = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES       = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

print(f"✅ Found {len(TRAINING_FILENAMES)} train, {len(VALIDATION_FILENAMES)} val, {len(TEST_FILENAMES)} test files")

# ====================================================
# CLASS LABELS
# ====================================================
CLASSES = [
    'pink primrose','hard-leaved pocket orchid','canterbury bells','sweet pea','wild geranium',
    'tiger lily','moon orchid','bird of paradise','monkshood','globe thistle','snapdragon',"colt's foot",
    'king protea','spear thistle','yellow iris','globe-flower','purple coneflower','peruvian lily','balloon flower',
    'giant white arum lily','fire lily','pincushion flower','fritillary','red ginger','grape hyacinth','corn poppy',
    'prince of wales feathers','stemless gentian','artichoke','sweet william','carnation','garden phlox',
    'love in the mist','cosmos','alpine sea holly','ruby-lipped cattleya','cape flower','great masterwort',
    'siam tulip','lenten rose','barberton daisy','daffodil','sword lily','poinsettia','bolero deep blue',
    'wallflower','marigold','buttercup','daisy','common dandelion','petunia','wild pansy','primula','sunflower',
    'lilac hibiscus','bishop of llandaff','gaura','geranium','orange dahlia','pink-yellow dahlia','cautleya spicata',
    'japanese anemone','black-eyed susan','silverbush','californian poppy','osteospermum','spring crocus','iris',
    'windflower','tree poppy','gazania','azalea','water lily','rose','thorn apple','morning glory','passion flower',
    'lotus','toad lily','anthurium','frangipani','clematis','hibiscus','columbine','desert-rose','tree mallow',
    'magnolia','cyclamen ','watercress','canna lily','hippeastrum ','bee balm','pink quill','foxglove',
    'bougainvillea','camellia','mallow','mexican petunia','bromelia','blanket flower','trumpet creeper',
    'blackberry lily','common tulip','wild rose'
]
NUM_CLASSES = len(CLASSES)
print(f"🌸 Total flower classes: {NUM_CLASSES}")

# TFRecord Dataset Loading

Universal TFRecord loader (for train, val, test):

In [ ]:
%%time
TRAIN_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALID_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES  = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

# Universal TFRecord Loader¶

In [ ]:
%%time
# ====================================================
# TFRecord Parsing & Dataset Loader
# ====================================================

AUTO = tf.data.AUTOTUNE

def decode_image(image_data):
    """Decode image bytes and normalize to [0,1]."""
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def read_tfrecord(example, labeled=True):
    """Parse TFRecord example into (image, label) or (image, id)."""
    if labeled:
        tfrecord_format = {
            "image": tf.io.FixedLenFeature([], tf.string),
            "class": tf.io.FixedLenFeature([], tf.int64),
            "id": tf.io.FixedLenFeature([], tf.string),
        }
    else:
        tfrecord_format = {
            "image": tf.io.FixedLenFeature([], tf.string),
            "id": tf.io.FixedLenFeature([], tf.string),
        }

    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])

    if labeled:
        label = tf.cast(example["class"], tf.int32)
        return image, label
    else:
        idnum = example["id"]
        return image, idnum

def load_dataset(filenames, labeled=True):
    """Load TFRecord dataset for train/val/test."""
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.map(lambda x: read_tfrecord(x, labeled), num_parallel_calls=AUTO)
    return dataset

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.shuffle(2048)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False)
    dataset = dataset.batch(BATCH_SIZE)
    if not ordered:
        options = tf.data.Options()
        options.experimental_deterministic = False
        dataset = dataset.with_options(options)
    dataset = dataset.prefetch(AUTO)
    return dataset


# Visual Check (Optional but Important)

In [ ]:
%%time
import matplotlib.pyplot as plt

train_ds = get_training_dataset()
sample_images, sample_labels = next(iter(train_ds.take(1)))

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(sample_images[i].numpy())
    label_index = sample_labels[i].numpy()
    plt.title(CLASSES[label_index])
    plt.axis("off")
plt.show()

# Model Preparation (GPU-based EfficientNetB0).

### EfficientNet B

In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

from tensorflow.keras import layers, models

In [ ]:
%%time
# ====================================================
# STEP 2 — MODEL PREPARATION (TFRecord format, GPU)
# ====================================================

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os

# Paths
DATA_DIR = "/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224"
IMAGE_SIZE = [224, 224]
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE

# ====================================================
# TFRecord utilities
# ====================================================

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def read_tfrecord(example, labeled=True):
    if labeled:
        tfrecord_format = {
            "image": tf.io.FixedLenFeature([], tf.string),
            "class": tf.io.FixedLenFeature([], tf.int64),
        }
        example = tf.io.parse_single_example(example, tfrecord_format)
        image = decode_image(example["image"])
        label = tf.cast(example["class"], tf.int32)
        return image, label
    else:
        tfrecord_format = {"image": tf.io.FixedLenFeature([], tf.string)}
        example = tf.io.parse_single_example(example, tfrecord_format)
        image = decode_image(example["image"])
        return image

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda x: read_tfrecord(x, labeled), num_parallel_calls=AUTO)
    return dataset

# ====================================================
# Load TFRecord files
# ====================================================

TRAINING_FILENAMES = tf.io.gfile.glob(f"{DATA_DIR}/train/*.tfrec")
VALIDATION_FILENAMES = tf.io.gfile.glob(f"{DATA_DIR}/val/*.tfrec")
TEST_FILENAMES = tf.io.gfile.glob(f"{DATA_DIR}/test/*.tfrec")

print("Train TFRecords:", len(TRAINING_FILENAMES))
print("Val TFRecords:", len(VALIDATION_FILENAMES))
print("Test TFRecords:", len(TEST_FILENAMES))

# ====================================================
# Create datasets
# ====================================================

train_ds = (
    load_dataset(TRAINING_FILENAMES, labeled=True)
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

val_ds = (
    load_dataset(VALIDATION_FILENAMES, labeled=True)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# ====================================================
# Visual check — display few flowers
# ====================================================

for imgs, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(imgs[i])
        plt.title(f"Label: {labels[i].numpy()}")
        plt.axis("off")
    plt.show()

# ====================================================
# Build EfficientNetB0 model
# ====================================================

# ====================================================
# Build EfficientNetB0 model (fixed Sequential version)
# ====================================================

from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

base_model = EfficientNetB0(include_top=False, input_shape=IMAGE_SIZE + [3], weights="imagenet")
base_model.trainable = False

# wrap preprocess_input into a Lambda layer
model = keras.Sequential([
    layers.Lambda(preprocess_input, name="preprocessing"),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(104, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# Training + Saving the Best Model (best_model.h5).

In [ ]:
%%time
# ====================================================
# STEP 3 — TRAINING EfficientNetB0 BASELINE
# ====================================================

from tensorflow.keras import callbacks

# Callbacks: Save best model & reduce LR if plateau
checkpoint_cb = callbacks.ModelCheckpoint(
    "best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

earlystop_cb = callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_cb = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=1,
    verbose=1,
    min_lr=1e-6
)

# ====================================================
# Train the model
# ====================================================

EPOCHS = 5  # (you can increase later if TPU time allows)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb, reduce_lr_cb],
    verbose=1
)

# ====================================================
# Visualize training progress
# ====================================================

import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend()
plt.title("Loss")

plt.show()

print("✅ Training finished. Best model saved as best_model.h5")

## ✅ Step 1: Reload best model

In [ ]:
%%time
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input

MODEL_PATH = "/kaggle/working/best_model.h5"

# Dummy function placeholder to avoid preprocess_input loading issue
def preprocess_input_fn(x): 
    return preprocess_input(x)

model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={"preprocess_input": preprocess_input_fn}
)
print("✅ Model loaded successfully from", MODEL_PATH)

## ✅ Step 2: Load test TFRecords

In [ ]:
%%time
TEST_FILENAMES = tf.io.gfile.glob("/kaggle/input/tpu-getting-started/tfrecords-jpeg-224x224/test/*.tfrec")
AUTO = tf.data.AUTOTUNE
IMAGE_SIZE = [224, 224]
BATCH_SIZE = 32

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def read_unlabeled_tfrecord(example):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    idnum = example["id"]
    return image, idnum

def load_dataset(filenames, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

test_dataset = load_dataset(TEST_FILENAMES, ordered=True).batch(BATCH_SIZE)
print("✅ Test dataset ready.")


## ✅ Step 3: Make predictions & create submission

In [ ]:
%%time
import pandas as pd
import numpy as np

image_ids = []
preds = []

for imgs, ids in test_dataset:
    p = model.predict(imgs, verbose=0)
    preds.extend(np.argmax(p, axis=-1))
    image_ids.extend([id.decode("utf-8") for id in ids.numpy()])

submission = pd.DataFrame({"id": image_ids, "label": preds})
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("✅ Submission file saved:", submission.shape)
submission.head(10)